# BindsNet Neuron (Node) Models

## 1. Table of Contents
1. Table of Contents
2. Overview
2. Import Statements
3. Neuron Models
    1. Integrate and Fire (IF)
    2. Leaky Integrate and Fire (LIF)

## 2. Overview
Ut prosim.

## 2. Import Statements

In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

from time import time as t

from bindsnet.encoding import *

# 3a. Integrate and Fire (IF) Neuron Model